# Process New Fire Data

This section decidedly does *not* use Mojo, as Mojo currently lacks useful I/O libraries.

In [166]:
import pandas as pd

In [167]:
raw_df = pd.read_csv("../data/gen_2/unprocessed/fire_archive_M-C61_28737.csv")

raw_df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-18.6804,145.5470,317.3,2.9,1.6,2000-11-01,16,Terra,MODIS,54,6.03,299.4,32.8,D,0
1,-18.4005,144.9007,318.8,3.2,1.7,2000-11-01,16,Terra,MODIS,64,6.03,301.5,42.6,D,0
2,-18.4459,144.8904,324.6,3.2,1.7,2000-11-01,16,Terra,MODIS,74,6.03,303.1,77.4,D,0
3,-18.4401,144.8603,317.7,3.3,1.7,2000-11-01,16,Terra,MODIS,61,6.03,303.2,38.5,D,0
4,-17.0518,143.8872,322.1,4.1,1.9,2000-11-01,16,Terra,MODIS,31,6.03,298.9,72.3,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571327,-19.4893,123.8846,306.3,1.1,1.1,2023-01-31,1726,Aqua,MODIS,67,6.03,290.5,6.1,N,0
5571328,-19.4908,123.8953,352.9,1.1,1.1,2023-01-31,1726,Aqua,MODIS,100,6.03,292.2,79.8,N,0
5571329,-19.4937,123.9167,332.9,1.1,1.1,2023-01-31,1726,Aqua,MODIS,100,6.03,292.1,36.0,N,0
5571330,-19.4965,123.9382,306.5,1.1,1.1,2023-01-31,1726,Aqua,MODIS,55,6.03,291.5,6.0,N,0


In [168]:
filtered_df = raw_df[raw_df['confidence'] > 80]

filtered_df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
29,-20.6460,148.4702,332.3,1.6,1.2,2000-11-01,17,Terra,MODIS,81,6.03,301.2,42.9,D,0
32,-20.4475,148.5904,332.7,1.6,1.2,2000-11-01,17,Terra,MODIS,82,6.03,299.9,46.4,D,0
49,-20.5602,146.0194,331.7,2.4,1.5,2000-11-01,17,Terra,MODIS,81,6.03,308.2,68.8,D,0
50,-20.5662,146.0241,332.1,2.4,1.5,2000-11-01,17,Terra,MODIS,82,6.03,306.5,66.9,D,0
52,-22.0605,145.5092,340.6,2.4,1.5,2000-11-01,17,Terra,MODIS,87,6.03,310.8,105.6,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571322,-31.3319,147.8329,312.7,1.4,1.2,2023-01-31,1550,Aqua,MODIS,86,6.03,291.5,19.1,N,0
5571326,-19.4922,123.9060,380.6,1.1,1.1,2023-01-31,1726,Aqua,MODIS,100,6.03,294.7,183.9,N,0
5571328,-19.4908,123.8953,352.9,1.1,1.1,2023-01-31,1726,Aqua,MODIS,100,6.03,292.2,79.8,N,0
5571329,-19.4937,123.9167,332.9,1.1,1.1,2023-01-31,1726,Aqua,MODIS,100,6.03,292.1,36.0,N,0


In [169]:
grouped_df = filtered_df.groupby("acq_date")

with_fire_area = pd.DataFrame(grouped_df.apply(lambda x: (x['scan'] * x['track']).sum()))
with_fire_area = with_fire_area.reset_index()
with_fire_area = with_fire_area.rename(columns={0: "fire_area"})

with_fire_area

,acq_date,fire_area
0,2000-11-01,550.38
1,2000-11-02,858.08
2,2000-11-03,1054.32
3,2000-11-04,931.25
4,2000-11-05,314.20
...,...,...
8091,2023-01-27,178.20
8092,2023-01-28,33.61
8093,2023-01-29,43.97
8094,2023-01-30,25.91


In [170]:
soi_df = pd.read_csv("../data/gen_2/unprocessed/soi_bom.csv", dtype={'month': str})

def get_soi(row):
    month = "".join(row['acq_date'].split('-')[0:2])
    soi = soi_df.loc[soi_df['month'] == month]['soi'].item()
    return soi

add_soi = with_fire_area.copy(deep=True)

add_soi['soi'] = with_fire_area.apply(get_soi, axis=1)

add_soi

,acq_date,fire_area,soi
0,2000-11-01,550.38,22.4
1,2000-11-02,858.08,22.4
2,2000-11-03,1054.32,22.4
3,2000-11-04,931.25,22.4
4,2000-11-05,314.20,22.4
...,...,...,...
8091,2023-01-27,178.20,11.8
8092,2023-01-28,33.61,11.8
8093,2023-01-29,43.97,11.8
8094,2023-01-30,25.91,11.8


In [171]:
# Intermediate save to run some pre-analysis
add_soi.to_csv("../data/gen_2/processed/with_fire_area_and_soi.csv", index=False)

This pre-analysis determined that adding the temperatures of various cities around Australia was necessary.

In [172]:
brisbane_df = pd.read_csv("../data/gen_2/unprocessed/temperature/Brisbane.csv", dtype={'Month': str, 'Day': str, 'Year': str})
melbourne_df = pd.read_csv("../data/gen_2/unprocessed/temperature/Melbourne.csv", dtype={'Month': str, 'Day': str, 'Year': str})
cairns_df = pd.read_csv("../data/gen_2/unprocessed/temperature/Cairns.csv", dtype={'Month': str, 'Day': str, 'Year': str})
perth_df = pd.read_csv("../data/gen_2/unprocessed/temperature/Perth.csv", dtype={'Month': str, 'Day': str, 'Year': str})
sydney_df = pd.read_csv("../data/gen_2/unprocessed/temperature/Sydney.csv", dtype={'Month': str, 'Day': str, 'Year': str})

In [173]:
brisbane_df['Month'] = brisbane_df['Month'].apply(lambda x: x.zfill(2))
brisbane_df['Day'] = brisbane_df['Day'].apply(lambda x: x.zfill(2))
melbourne_df['Month'] = melbourne_df['Month'].apply(lambda x: x.zfill(2))
melbourne_df['Day'] = melbourne_df['Day'].apply(lambda x: x.zfill(2))
cairns_df['Month'] = cairns_df['Month'].apply(lambda x: x.zfill(2))
cairns_df['Day'] = cairns_df['Day'].apply(lambda x: x.zfill(2))
perth_df['Month'] = perth_df['Month'].apply(lambda x: x.zfill(2))
perth_df['Day'] = perth_df['Day'].apply(lambda x: x.zfill(2))
sydney_df['Month'] = sydney_df['Month'].apply(lambda x: x.zfill(2))
sydney_df['Day'] = sydney_df['Day'].apply(lambda x: x.zfill(2))

brisbane_df

,Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality
0,IDCJAC0010,40913,1999,01,01,NaN,NaN,NaN
1,IDCJAC0010,40913,1999,01,02,NaN,NaN,NaN
2,IDCJAC0010,40913,1999,01,03,NaN,NaN,NaN
3,IDCJAC0010,40913,1999,01,04,NaN,NaN,NaN
4,IDCJAC0010,40913,1999,01,05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9109,IDCJAC0010,40913,2023,12,10,29.9,1.0,N
9110,IDCJAC0010,40913,2023,12,11,29.3,1.0,N
9111,IDCJAC0010,40913,2023,12,12,30.5,1.0,N
9112,IDCJAC0010,40913,2023,12,13,30.6,1.0,N


In [174]:
columns = ['Year', 'Month', 'Day']

brisbane_df['date'] = brisbane_df[columns].apply(lambda row: '-'.join(row.values), axis=1)
melbourne_df['date'] = melbourne_df[columns].apply(lambda row: '-'.join(row.values), axis=1)
cairns_df['date'] = cairns_df[columns].apply(lambda row: '-'.join(row.values), axis=1)
perth_df['date'] = perth_df[columns].apply(lambda row: '-'.join(row.values), axis=1)
sydney_df['date'] = sydney_df[columns].apply(lambda row: '-'.join(row.values), axis=1)

brisbane_df

,Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality,date
0,IDCJAC0010,40913,1999,01,01,NaN,NaN,NaN,1999-01-01
1,IDCJAC0010,40913,1999,01,02,NaN,NaN,NaN,1999-01-02
2,IDCJAC0010,40913,1999,01,03,NaN,NaN,NaN,1999-01-03
3,IDCJAC0010,40913,1999,01,04,NaN,NaN,NaN,1999-01-04
4,IDCJAC0010,40913,1999,01,05,NaN,NaN,NaN,1999-01-05
...,...,...,...,...,...,...,...,...,...
9109,IDCJAC0010,40913,2023,12,10,29.9,1.0,N,2023-12-10
9110,IDCJAC0010,40913,2023,12,11,29.3,1.0,N,2023-12-11
9111,IDCJAC0010,40913,2023,12,12,30.5,1.0,N,2023-12-12
9112,IDCJAC0010,40913,2023,12,13,30.6,1.0,N,2023-12-13


In [175]:
brisbane_df = brisbane_df[['date', 'Maximum temperature (Degree C)']]
brisbane_df = brisbane_df.rename(columns={'Maximum temperature (Degree C)': 'max_t_bne'})
melbourne_df = melbourne_df[['date', 'Maximum temperature (Degree C)']]
melbourne_df = melbourne_df.rename(columns={'Maximum temperature (Degree C)': 'max_t_mel'})
cairns_df = cairns_df[['date', 'Maximum temperature (Degree C)']]
cairns_df = cairns_df.rename(columns={'Maximum temperature (Degree C)': 'max_t_cns'})
perth_df = perth_df[['date', 'Maximum temperature (Degree C)']]
perth_df = perth_df.rename(columns={'Maximum temperature (Degree C)': 'max_t_pth'})
sydney_df = sydney_df[['date', 'Maximum temperature (Degree C)']]
sydney_df = sydney_df.rename(columns={'Maximum temperature (Degree C)': 'max_t_syd'})

brisbane_df

,date,max_t_bne
0,1999-01-01,NaN
1,1999-01-02,NaN
2,1999-01-03,NaN
3,1999-01-04,NaN
4,1999-01-05,NaN
...,...,...
9109,2023-12-10,29.9
9110,2023-12-11,29.3
9111,2023-12-12,30.5
9112,2023-12-13,30.6


In [176]:
# Find how many NaN rows in each
print(brisbane_df.isna().sum())
print(melbourne_df.isna().sum())
print(cairns_df.isna().sum())
print(perth_df.isna().sum())
print(sydney_df.isna().sum())

date           0
max_t_bne    380
dtype: int64
date           0
max_t_mel    184
dtype: int64
date           0
max_t_cns    360
dtype: int64
date          0
max_t_pth    12
dtype: int64
date           0
max_t_syd    115
dtype: int64


In [177]:
weather_dataframes = [brisbane_df, melbourne_df, cairns_df, perth_df, sydney_df]
weather_dataframes = [df.set_index('date') for df in weather_dataframes]

weather_dataframe = weather_dataframes[0].join(weather_dataframes[1:])

weather_dataframe = weather_dataframe.reset_index()

weather_dataframe

,date,max_t_bne,max_t_mel,max_t_cns,max_t_pth,max_t_syd
0,1999-01-01,NaN,26.1,30.9,NaN,25.7
1,1999-01-02,NaN,36.3,31.6,30.2,28.3
2,1999-01-03,NaN,25.9,31.2,28.0,29.8
3,1999-01-04,NaN,26.4,32.4,23.4,30.9
4,1999-01-05,NaN,39.4,32.2,22.4,31.1
...,...,...,...,...,...,...
9109,2023-12-10,29.9,19.4,NaN,NaN,NaN
9110,2023-12-11,29.3,19.0,NaN,NaN,NaN
9111,2023-12-12,30.5,32.9,NaN,NaN,NaN
9112,2023-12-13,30.6,33.5,NaN,NaN,NaN


In [178]:
add_soi.rename(columns={'acq_date': 'date'}, inplace=True)

add_soi.drop_duplicates(subset=['date'], inplace=True)
add_soi.dropna()

,date,fire_area,soi
0,2000-11-01,550.38,22.4
1,2000-11-02,858.08,22.4
2,2000-11-03,1054.32,22.4
3,2000-11-04,931.25,22.4
4,2000-11-05,314.20,22.4
...,...,...,...
8091,2023-01-27,178.20,11.8
8092,2023-01-28,33.61,11.8
8093,2023-01-29,43.97,11.8
8094,2023-01-30,25.91,11.8


In [180]:
add_weather = add_soi.merge(weather_dataframe, on="date")

add_weather

,date,fire_area,soi,max_t_bne,max_t_mel,max_t_cns,max_t_pth,max_t_syd
0,2000-11-01,550.38,22.4,27.0,19.1,31.1,24.2,22.1
1,2000-11-02,858.08,22.4,24.0,22.7,31.7,21.6,22.1
2,2000-11-03,1054.32,22.4,26.0,19.1,34.9,22.4,24.2
3,2000-11-04,931.25,22.4,28.0,17.0,30.4,27.6,21.1
4,2000-11-05,314.20,22.4,28.0,17.0,30.8,28.9,20.0
...,...,...,...,...,...,...,...,...
8091,2023-01-27,178.20,11.8,33.4,28.9,32.3,27.2,25.9
8092,2023-01-28,33.61,11.8,32.9,34.9,31.9,31.3,31.0
8093,2023-01-29,43.97,11.8,32.1,23.9,32.3,38.4,30.6
8094,2023-01-30,25.91,11.8,32.3,21.9,31.1,38.1,30.7


In [183]:
add_weather = add_weather.dropna()

In [184]:
add_weather.to_csv("../data/gen_2/processed/with_fire_area_and_soi_and_weather.csv", index=False)

In [186]:
import numpy as np

add_sin_cos_df = add_weather.copy(deep=True)
add_sin_cos_df = add_sin_cos_df.reset_index()

add_sin_cos_df

,index,date,fire_area,soi,max_t_bne,max_t_mel,max_t_cns,max_t_pth,max_t_syd
0,0,2000-11-01,550.38,22.4,27.0,19.1,31.1,24.2,22.1
1,1,2000-11-02,858.08,22.4,24.0,22.7,31.7,21.6,22.1
2,2,2000-11-03,1054.32,22.4,26.0,19.1,34.9,22.4,24.2
3,3,2000-11-04,931.25,22.4,28.0,17.0,30.4,27.6,21.1
4,4,2000-11-05,314.20,22.4,28.0,17.0,30.8,28.9,20.0
...,...,...,...,...,...,...,...,...,...
8036,8091,2023-01-27,178.20,11.8,33.4,28.9,32.3,27.2,25.9
8037,8092,2023-01-28,33.61,11.8,32.9,34.9,31.9,31.3,31.0
8038,8093,2023-01-29,43.97,11.8,32.1,23.9,32.3,38.4,30.6
8039,8094,2023-01-30,25.91,11.8,32.3,21.9,31.1,38.1,30.7


In [187]:
add_sin_cos_df['sin_signal'] = np.sin(pd.to_datetime(add_sin_cos_df['date']).dt.day_of_year / 365 * 2 * np.pi)
add_sin_cos_df['cos_signal'] = np.cos(pd.to_datetime(add_sin_cos_df['date']).dt.day_of_year / 365 * 2 * np.pi)

add_sin_cos_df

,index,date,fire_area,soi,max_t_bne,max_t_mel,max_t_cns,max_t_pth,max_t_syd,sin_signal,cos_signal
0,0,2000-11-01,550.38,22.4,27.0,19.1,31.1,24.2,22.1,-0.849817,0.527078
1,1,2000-11-02,858.08,22.4,24.0,22.7,31.7,21.6,22.1,-0.840618,0.541628
2,2,2000-11-03,1054.32,22.4,26.0,19.1,34.9,22.4,24.2,-0.831171,0.556017
3,3,2000-11-04,931.25,22.4,28.0,17.0,30.4,27.6,21.1,-0.821477,0.570242
4,4,2000-11-05,314.20,22.4,28.0,17.0,30.8,28.9,20.0,-0.811539,0.584298
...,...,...,...,...,...,...,...,...,...,...,...
8036,8091,2023-01-27,178.20,11.8,33.4,28.9,32.3,27.2,25.9,0.448229,0.893919
8037,8092,2023-01-28,33.61,11.8,32.9,34.9,31.9,31.3,31.0,0.463550,0.886071
8038,8093,2023-01-29,43.97,11.8,32.1,23.9,32.3,38.4,30.6,0.478734,0.877960
8039,8094,2023-01-30,25.91,11.8,32.3,21.9,31.1,38.1,30.7,0.493776,0.869589


In [188]:
add_year_df = add_sin_cos_df.copy(deep=True)
add_year_df['year'] = add_year_df['date'].apply(lambda x: x.split('-')[0])

add_year_df

,index,date,fire_area,soi,max_t_bne,max_t_mel,max_t_cns,max_t_pth,max_t_syd,sin_signal,cos_signal,year
0,0,2000-11-01,550.38,22.4,27.0,19.1,31.1,24.2,22.1,-0.849817,0.527078,2000
1,1,2000-11-02,858.08,22.4,24.0,22.7,31.7,21.6,22.1,-0.840618,0.541628,2000
2,2,2000-11-03,1054.32,22.4,26.0,19.1,34.9,22.4,24.2,-0.831171,0.556017,2000
3,3,2000-11-04,931.25,22.4,28.0,17.0,30.4,27.6,21.1,-0.821477,0.570242,2000
4,4,2000-11-05,314.20,22.4,28.0,17.0,30.8,28.9,20.0,-0.811539,0.584298,2000
...,...,...,...,...,...,...,...,...,...,...,...,...
8036,8091,2023-01-27,178.20,11.8,33.4,28.9,32.3,27.2,25.9,0.448229,0.893919,2023
8037,8092,2023-01-28,33.61,11.8,32.9,34.9,31.9,31.3,31.0,0.463550,0.886071,2023
8038,8093,2023-01-29,43.97,11.8,32.1,23.9,32.3,38.4,30.6,0.478734,0.877960,2023
8039,8094,2023-01-30,25.91,11.8,32.3,21.9,31.1,38.1,30.7,0.493776,0.869589,2023


In [189]:
# Drop blank
add_year_df = add_year_df.dropna()

add_year_df

,index,date,fire_area,soi,max_t_bne,max_t_mel,max_t_cns,max_t_pth,max_t_syd,sin_signal,cos_signal,year
0,0,2000-11-01,550.38,22.4,27.0,19.1,31.1,24.2,22.1,-0.849817,0.527078,2000
1,1,2000-11-02,858.08,22.4,24.0,22.7,31.7,21.6,22.1,-0.840618,0.541628,2000
2,2,2000-11-03,1054.32,22.4,26.0,19.1,34.9,22.4,24.2,-0.831171,0.556017,2000
3,3,2000-11-04,931.25,22.4,28.0,17.0,30.4,27.6,21.1,-0.821477,0.570242,2000
4,4,2000-11-05,314.20,22.4,28.0,17.0,30.8,28.9,20.0,-0.811539,0.584298,2000
...,...,...,...,...,...,...,...,...,...,...,...,...
8036,8091,2023-01-27,178.20,11.8,33.4,28.9,32.3,27.2,25.9,0.448229,0.893919,2023
8037,8092,2023-01-28,33.61,11.8,32.9,34.9,31.9,31.3,31.0,0.463550,0.886071,2023
8038,8093,2023-01-29,43.97,11.8,32.1,23.9,32.3,38.4,30.6,0.478734,0.877960,2023
8039,8094,2023-01-30,25.91,11.8,32.3,21.9,31.1,38.1,30.7,0.493776,0.869589,2023


In [190]:
add_year_df.describe()

,index,fire_area,soi,max_t_bne,max_t_mel,max_t_cns,max_t_pth,max_t_syd,sin_signal,cos_signal
count,8041.000000,8041.000000,8041.000000,8041.000000,8041.000000,8041.000000,8041.000000,8041.000000,8041.000000,8041.000000
mean,4035.780624,574.480246,1.162903,26.634797,20.470626,29.585549,24.933789,23.264557,-0.004822,0.011100
std,2335.884970,816.720763,10.127510,3.823670,6.581535,2.546262,6.094557,5.033739,0.704686,0.709504
min,0.000000,1.000000,-28.600000,12.600000,8.100000,20.000000,11.500000,11.400000,-0.999991,-0.999963
25%,2013.000000,140.040000,-6.300000,23.800000,15.300000,27.700000,20.100000,19.500000,-0.705584,-0.696376
50%,4028.000000,305.110000,0.500000,26.900000,19.200000,29.700000,23.800000,22.900000,-0.008607,0.021516
75%,6050.000000,670.210000,8.900000,29.400000,24.400000,31.500000,29.200000,26.500000,0.699458,0.726608
max,8095.000000,13241.020000,27.100000,41.700000,46.800000,42.600000,44.400000,46.400000,0.999991,1.000000


In [191]:
add_year_df.to_csv("../data/gen_2/processed/with_fire_area_soi_weather_time.csv", index=False)